# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/drive/MyDrive/Assignments/Udacity_NLP/2.7-Machine-Translation/* .

Mounted at /content/drive


In [2]:
%load_ext autoreload
# %aimport helper, tests
%autoreload 1

In [3]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3553178451838192439
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14444920832
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11197109769465750475
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [5]:
# Load English data
english_sentences = helper.load_data('/content/data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('/content/data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [6]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [8]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizeor used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer(
        num_words=None,
        # filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
        lower=True,
        split=' ',
        char_level=False,
        oov_token=None
    )
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [9]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding='post')
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [11]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [12]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 1e-3
    unit_size = 256
    input_sequence = Input(input_shape[1:])
    rnn = GRU(unit_size, return_sequences=True)(input_sequence)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_sequence, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
# tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
simple_rnn_tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
simple_rnn_tmp_x = simple_rnn_tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    simple_rnn_tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)
simple_rnn_model.fit(simple_rnn_tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# # Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(simple_rnn_tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 9s 26ms/step - loss: 2.5619 - accuracy: 0.4766 - val_loss: 1.9139 - val_accuracy: 0.5538
Epoch 2/10
108/108 [==============================] - 2s 22ms/step - loss: 1.6681 - accuracy: 0.5813 - val_loss: 1.5169 - val_accuracy: 0.5979
Epoch 3/10
108/108 [==============================] - 2s 22ms/step - loss: 1.4352 - accuracy: 0.6144 - val_loss: 1.3605 - val_accuracy: 0.6249
Epoch 4/10
108/108 [==============================] - 2s 22ms/step - loss: 1.3023 - accuracy: 0.6361 - val_loss: 1.2444 - val_accuracy: 0.6464
Epoch 5/10
108/108 [==============================] - 2s 22ms/step - loss: 1.2052 - accuracy: 0.6534 - val_loss: 1.1687 - val_accuracy: 0.6612
Epoch 6/10
108/108 [==============================] - 2s 22ms/step - loss: 1.1425 - accuracy: 0.6614 - val_loss: 1.1152 - val_accuracy: 0.6629
Epoch 7/10
108/108 [==============================] - 2s 22ms/step - loss: 1.0984 - accuracy: 0.6679 - val_loss: 1.0789 - val_accuracy: 0.6689

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [13]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    embedding_size = 300
    unit_size = 256
    learning_rate = 1e-3
    input_sequence = Input(input_shape[1:])
    embedding = Embedding(english_vocab_size, embedding_size)(input_sequence)
    rnn = GRU(unit_size, return_sequences=True)(embedding)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_sequence, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
# tests.test_embed_model(embed_model)

# TODO: Reshape the input
embed_rnn_tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
embed_rnn_tmp_x = embed_rnn_tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train the neural network
embed_rnn_model = embed_model(
    embed_rnn_tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)
embed_rnn_model.fit(embed_rnn_tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# # TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(embed_rnn_tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 5s 34ms/step - loss: 2.9411 - accuracy: 0.4582 - val_loss: 2.0069 - val_accuracy: 0.5542
Epoch 2/10
108/108 [==============================] - 4s 34ms/step - loss: 1.4311 - accuracy: 0.6638 - val_loss: 1.0375 - val_accuracy: 0.7495
Epoch 3/10
108/108 [==============================] - 3s 30ms/step - loss: 0.8400 - accuracy: 0.7806 - val_loss: 0.6839 - val_accuracy: 0.8079
Epoch 4/10
108/108 [==============================] - 3s 30ms/step - loss: 0.5894 - accuracy: 0.8302 - val_loss: 0.5118 - val_accuracy: 0.8472
Epoch 5/10
108/108 [==============================] - 3s 30ms/step - loss: 0.4646 - accuracy: 0.8600 - val_loss: 0.4268 - val_accuracy: 0.8686
Epoch 6/10
108/108 [==============================] - 3s 30ms/step - loss: 0.3908 - accuracy: 0.8789 - val_loss: 0.3597 - val_accuracy: 0.8878
Epoch 7/10
108/108 [==============================] - 3s 30ms/step - loss: 0.3389 - accuracy: 0.8937 - val_loss: 0.3191 - val_accuracy: 0.8998

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [14]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    embedding_size = 300
    unit_size = 256
    learning_rate = 1e-3
    input_sequence = Input(input_shape[1:])
    bidirectional = Bidirectional(GRU(unit_size, return_sequences=True, activation='relu'))(input_sequence)
    logits = TimeDistributed(Dense(french_vocab_size))(bidirectional)
    model = Model(input_sequence, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
# tests.test_bd_model(bd_model)

# TODO: Train and Print prediction(s)
bd_rnn_tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
bd_rnn_tmp_x = bd_rnn_tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# TODO: Train the neural network
bd_rnn_model = bd_model(
    bd_rnn_tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)
bd_rnn_model.fit(bd_rnn_tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# # TODO: Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(bd_rnn_tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 14s 107ms/step - loss: 2.0558 - accuracy: 0.5514 - val_loss: 1.3369 - val_accuracy: 0.6213
Epoch 2/10
108/108 [==============================] - 12s 108ms/step - loss: 1.2225 - accuracy: 0.6416 - val_loss: 1.2213 - val_accuracy: 0.6310
Epoch 3/10
108/108 [==============================] - 12s 107ms/step - loss: 1.1082 - accuracy: 0.6635 - val_loss: 1.0577 - val_accuracy: 0.6761
Epoch 4/10
108/108 [==============================] - 11s 106ms/step - loss: 1.0332 - accuracy: 0.6787 - val_loss: 1.0082 - val_accuracy: 0.6847
Epoch 5/10
108/108 [==============================] - 11s 104ms/step - loss: 0.9845 - accuracy: 0.6874 - val_loss: 0.9674 - val_accuracy: 0.6902
Epoch 6/10
108/108 [==============================] - 13s 118ms/step - loss: 0.9439 - accuracy: 0.6950 - val_loss: 0.9269 - val_accuracy: 0.6951
Epoch 7/10
108/108 [==============================] - 11s 104ms/step - loss: 0.9236 - accuracy: 0.6983 - val_loss: 0.9003 - val_ac

### Model 4: Encoder-Decoder (IMPLEMENTATION)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [15]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    embedding_size = 300
    unit_size = 256
    learning_rate = 1e-3
    input_sequence = Input(input_shape[1:])
    enc_output = Bidirectional(GRU(unit_size, return_sequences=False, activation='relu'))(input_sequence)
    enc_repeat = RepeatVector(output_sequence_length)(enc_output)
    dec_output= Bidirectional(GRU(unit_size, return_sequences=True, activation='relu'))(enc_repeat)
    logits = TimeDistributed(Dense(french_vocab_size))(dec_output)
    model = Model(input_sequence, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
# tests.test_encdec_model(encdec_model)

# TODO: Train and Print prediction(s)
encdec_rnn_tmp_x = pad(preproc_english_sentences, max_english_sequence_length)
encdec_rnn_tmp_x = encdec_rnn_tmp_x.reshape((-1, preproc_english_sentences.shape[-1], 1))

# TODO: Train the neural network
encdec_rnn_model = encdec_model(
    encdec_rnn_tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)

encdec_rnn_model.fit(encdec_rnn_tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# # TODO: Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(encdec_rnn_tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 27s 214ms/step - loss: 2.0924 - accuracy: 0.5224 - val_loss: 1.4130 - val_accuracy: 0.6024
Epoch 2/10
108/108 [==============================] - 22s 200ms/step - loss: 1.2958 - accuracy: 0.6246 - val_loss: 1.2311 - val_accuracy: 0.6360
Epoch 3/10
108/108 [==============================] - 22s 200ms/step - loss: 1.1487 - accuracy: 0.6511 - val_loss: 1.0889 - val_accuracy: 0.6658
Epoch 4/10
108/108 [==============================] - 22s 202ms/step - loss: 1.0771 - accuracy: 0.6664 - val_loss: 1.0244 - val_accuracy: 0.6804
Epoch 5/10
108/108 [==============================] - 22s 199ms/step - loss: 1.0123 - accuracy: 0.6789 - val_loss: 0.9893 - val_accuracy: 0.6791
Epoch 6/10
108/108 [==============================] - 23s 217ms/step - loss: 0.9662 - accuracy: 0.6871 - val_loss: 0.9301 - val_accuracy: 0.6948
Epoch 7/10
108/108 [==============================] - 22s 203ms/step - loss: 0.9105 - accuracy: 0.6986 - val_loss: 0.8825 - val_ac

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [16]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    embedding_size = 300
    unit_size = 256
    learning_rate = 1e-3
    input_sequence = Input(input_shape[1:])
    input_vector = Embedding(english_vocab_size, embedding_size)(input_sequence)
    enc_output = Bidirectional(GRU(unit_size, return_sequences=False, activation='relu'))(input_vector)
    enc_repeat = RepeatVector(output_sequence_length)(enc_output)
    dec_output= Bidirectional(GRU(unit_size, return_sequences=True, activation='relu'))(enc_repeat)
    logits = TimeDistributed(Dense(french_vocab_size))(dec_output)
    model = Model(input_sequence, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
# tests.test_model_final(model_final)

print('Final Model Loaded')
# TODO: Train the final model
# final_rnn_tmp_x = pad(preproc_english_sentences, max_english_sequence_length)
# final_rnn_tmp_x = final_rnn_tmp_x.reshape((-1, preproc_english_sentences.shape[-1]))

# TODO: Train the neural network
final_rnn_model = model_final(
    preproc_english_sentences.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)

final_rnn_model.fit(preproc_english_sentences, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# # # TODO: Print prediction(s)
print(logits_to_text(final_rnn_model.predict(preproc_english_sentences[:1])[0], french_tokenizer))

Final Model Loaded


Epoch 1/10
108/108 [==============================] - 28s 223ms/step - loss: 2.7078 - accuracy: 0.4632 - val_loss: 1.8357 - val_accuracy: 0.5458
Epoch 2/10
108/108 [==============================] - 23s 216ms/step - loss: 1.5286 - accuracy: 0.6052 - val_loss: 1.2206 - val_accuracy: 0.6653
Epoch 3/10
108/108 [==============================] - 25s 233ms/step - loss: 1.0500 - accuracy: 0.7055 - val_loss: 0.8758 - val_accuracy: 0.7429
Epoch 4/10
108/108 [==============================] - 23s 216ms/step - loss: 0.8705 - accuracy: 0.7448 - val_loss: 0.8591 - val_accuracy: 0.7435
Epoch 5/10
108/108 [==============================] - 23s 215ms/step - loss: 0.6647 - accuracy: 0.7983 - val_loss: 0.5888 - val_accuracy: 0.8193
Epoch 6/10
108/108 [==============================] - 24s 220ms/step - loss: 0.5186 - accuracy: 0.8385 - val_loss: 0.4823 - val_accuracy: 0.8497
Epoch 7/10
108/108 [==============================] - 25s 233ms/step - loss: 0.4024 - accuracy: 0.8717 - val_loss: 0.3611 - val_ac

new jersey est parfois calme en cours et l' automne est il en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Prediction (IMPLEMENTATION)

In [17]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    tmp_x = pad(x, x.shape[1])
    tmp_x = tmp_x.reshape((-1, x.shape[-1]))

    model = model_final(
        tmp_x.shape,
        y.shape[1],
        len(x_tk.word_index),
        len(y_tk.word_index)
    )
    model.fit(tmp_x, y, batch_size=1024, epochs=10, validation_split=0.2)
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Epoch 1/10
108/108 [==============================] - 28s 223ms/step - loss: 2.7858 - accuracy: 0.4593 - val_loss: nan - val_accuracy: 0.5344
Epoch 2/10
108/108 [==============================] - 24s 219ms/step - loss: 1.5225 - accuracy: 0.6034 - val_loss: nan - val_accuracy: 0.6615
Epoch 3/10
108/108 [==============================] - 23s 216ms/step - loss: 1.1125 - accuracy: 0.6912 - val_loss: nan - val_accuracy: 0.7291
Epoch 4/10
108/108 [==============================] - 25s 232ms/step - loss: 0.8668 - accuracy: 0.7456 - val_loss: nan - val_accuracy: 0.7666
Epoch 5/10
108/108 [==============================] - 24s 221ms/step - loss: 0.7133 - accuracy: 0.7840 - val_loss: nan - val_accuracy: 0.8063
Epoch 6/10
108/108 [==============================] - 23s 217ms/step - loss: 0.6564 - accuracy: 0.8021 - val_loss: nan - val_accuracy: 0.6917
Epoch 7/10
108/108 [==============================] - 24s 218ms/step - loss: 0.5663 - accuracy: 0.8252 - val_loss: nan - val_accuracy: 0.8678
Epoch 

Sample 1:
il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?

In [18]:
from sklearn.model_selection import train_test_split

def split_predictions(x, y, x_tk, y_tk):
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
  tmp_x = pad(X_train, X_train.shape[1])
  tmp_x = tmp_x.reshape((-1, X_train.shape[-1]))
  model = model_final(
      tmp_x.shape,
      y.shape[1],
      len(x_tk.word_index),
      len(y_tk.word_index))
  
  model.fit(tmp_x, y_train, batch_size=1024, epochs=10, validation_split=0.2)
  eval_score = model.evaluate(X_test, y_test, batch_size=256)
  print(eval_score)

split_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Epoch 1/10
87/87 [==============================] - 23s 220ms/step - loss: nan - accuracy: 0.4080 - val_loss: nan - val_accuracy: 0.4093
Epoch 2/10
87/87 [==============================] - 21s 236ms/step - loss: nan - accuracy: 0.4082 - val_loss: nan - val_accuracy: 0.4093
Epoch 3/10
87/87 [==============================] - 19s 213ms/step - loss: nan - accuracy: 0.4082 - val_loss: nan - val_accuracy: 0.4093
Epoch 4/10
87/87 [==============================] - 19s 214ms/step - loss: nan - accuracy: 0.4082 - val_loss: nan - val_accuracy: 0.4093
Epoch 5/10
87/87 [==============================] - 19s 215ms/step - loss: nan - accuracy: 0.4082 - val_loss: nan - val_accuracy: 0.4093
Epoch 6/10
87/87 [==============================] - 19s 214ms/step - loss: nan - accuracy: 0.4082 - val_loss: nan - val_accuracy: 0.4093
Epoch 7/10
87/87 [==============================] - 20s 234ms/step - loss: nan - accuracy: 0.4082 - val_loss: nan - val_accuracy: 0.4093
Epoch 8/10
87/87 [=======================